In [1]:
# import required libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tensorflow.python.framework import ops
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib
import argparse
import os
import sys
import time
import math
from six.moves import xrange
ops.reset_default_graph()

# Se cargan los datos

In [2]:
#sess = tf.Session()# Training loop
Xdf_raw = pd.read_csv("../../IndoorFingerprint.csv").drop("Y", axis=1)
Ydf_raw = pd.read_csv("../../IndoorFingerprint.csv").drop("X", axis=1)
x = tf.placeholder(tf.float32, shape=[None, 8])
y_ = tf.placeholder(tf.float32, shape=[None, 11])

# Se escalan los datos

In [3]:
# Normalizacion de los datos
atribute_numbers = Xdf_raw.shape[1]-1

# Valores para el dataset que predice X
Xtrx, Xtex, ytrx, ytex = train_test_split(
Xdf_raw.iloc[:, 1:], Xdf_raw["X"], test_size=0.2, random_state=0)
train_size = len(Xtrx)
test_size = len(Xtex)

# Valores para el dataset que predice Y
Xtry, Xtey, ytry, ytey = train_test_split(
Ydf_raw.iloc[:, 1:], Ydf_raw["Y"], test_size=0.2, random_state=0)

# Se normalizan los datos para ambos

scaler = preprocessing.StandardScaler()
Xtrx = scaler.fit_transform(Xtrx)
Xtex = scaler.transform(Xtex)

scaler = preprocessing.StandardScaler()
Xtry = scaler.fit_transform(Xtry)
Xtey = scaler.transform(Xtey)

# One Hot Vector

In [4]:
class_number_x = ytrx.unique().size
class_number_y = ytry.unique().size

ytrx_dummy = pd.get_dummies(ytrx)
ytex_dummy = pd.get_dummies(ytex)

ytry_dummy = pd.get_dummies(ytry)
ytey_dummy = pd.get_dummies(ytey)

# Se convierten valores a Tensores X

Xtrx = np.array(Xtrx,dtype='float32')
Xtex = np.array(Xtex,dtype='float32')
ytrx_dummy = np.array(ytrx_dummy,dtype='int32')
ytex_dummy = np.array(ytex_dummy,dtype='int32')

# Se convierten valores a Tensores Y

Xtry = np.array(Xtry,dtype='float32')
Xtey = np.array(Xtey,dtype='float32')
ytry_dummy = np.array(ytry_dummy,dtype='int32')
ytey_dummy = np.array(ytey_dummy,dtype='int32')

ytrx_cat = pd.Series(ytrx, dtype="category")
ytrx = np.array(ytrx_cat.cat.codes,dtype='int32')
ytex_cat = pd.Series(ytex, dtype="category")
ytex = np.array(ytex_cat.cat.codes,dtype='int32')

ytry_cat = pd.Series(ytry, dtype="category")
ytry = np.array(ytry_cat.cat.codes,dtype='int32')
ytey_cat = pd.Series(ytey, dtype="category")
ytey = np.array(ytey_cat.cat.codes,dtype='int32')

print(Xtrx[20])
print(ytrx[20])
print(Xtry[20])
print(ytry[20])

[ 2.80714798 -0.36167914 -1.0649991   4.10078526 -0.44053763  0.52955198
  0.41685054  1.31442308]
10
[ 2.80714798 -0.36167914 -1.0649991   4.10078526 -0.44053763  0.52955198
  0.41685054  1.31442308]
1


In [5]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [6]:
# Variables
learning_rate = 0.3
max_steps = 20000
hidden1 = 256
hidden2 = 64
batch_size = 32
input_data_dir = os.path.join(os.getenv('TEST_TMPDIR', '/tmp'),
                           'tensorflow/mnist/input_data'),
log_dir = os.path.join(os.getenv('TEST_TMPDIR', 'C:/Users/felip/OneDrive/Documentos/Universidad/Memoria/Complementos/Indoor-ML/CNN/CNN'),
                           'logs/fully_connected_feed')
fake_data = False

In [7]:
# The MNIST dataset has 10 classes, representing the digits 0 through 9.
#NUM_CLASSES = class_number_x

# The MNIST images are always 28x28 pixels.
IMAGE_SIZE = 28
IMAGE_PIXELS = atribute_numbers


def inference(images, hidden1_units, hidden2_units, NUM_CLASSES):
    """Build the MNIST model up to where it may be used for inference.
  Args:
    images: Images placeholder, from inputs().
    hidden1_units: Size of the first hidden layer.
    hidden2_units: Size of the second hidden layer.
  Returns:
    softmax_linear: Output tensor with the computed logits.
  """
  # Hidden 1
    with tf.name_scope('hidden1'):
        weights = tf.Variable(
            tf.truncated_normal([IMAGE_PIXELS, hidden1_units],
                                stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden1_units]),
                             name='biases')
        hidden1 = tf.nn.relu(tf.matmul(images, weights) + biases)
  # Hidden 2
    with tf.name_scope('hidden2'):
        weights = tf.Variable(
            tf.truncated_normal([hidden1_units, hidden2_units],
                                stddev=1.0 / math.sqrt(float(hidden1_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([hidden2_units]),
                             name='biases')
        hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
  # Linear
    with tf.name_scope('softmax_linear'):
        weights = tf.Variable(
            tf.truncated_normal([hidden2_units, NUM_CLASSES],
                                stddev=1.0 / math.sqrt(float(hidden2_units))),
            name='weights')
        biases = tf.Variable(tf.zeros([NUM_CLASSES]),
                             name='biases')
        logits = tf.matmul(hidden2, weights) + biases
    return logits


def lossF(logits, labels):
    """Calculates the loss from the logits and the labels.
  Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size].
  Returns:
    loss: Loss tensor of type float.
  """
    labels = tf.to_int64(labels)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
      labels=labels, logits=logits, name='xentropy')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='xentropy_mean')
    return cross_entropy_mean


def training(loss, learning_rate, mode):
    """Sets up the training Ops.
  Creates a summarizer to track the loss over time in TensorBoard.
  Creates an optimizer and applies the gradients to all trainable variables.
  The Op returned by this function is what must be passed to the
  `sess.run()` call to cause the model to train.
  Args:
    loss: Loss tensor, from loss().
    learning_rate: The learning rate to use for gradient descent.
  Returns:
    train_op: The Op for training.
  """
    # Add a scalar summary for the snapshot loss.
    if mode == 0:
        tf.summary.scalar('loss_x', loss)
    # Create the gradient descent optimizer with the given learning rate.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    # Create a variable to track the global step.
    global_step = tf.Variable(0, name='global_step', trainable=False)
    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single training step.
    train_op = optimizer.minimize(loss, global_step=global_step)
    return train_op


def evaluation(logits, labels):
    """Evaluate the quality of the logits at predicting the label.
  Args:
    logits: Logits tensor, float - [batch_size, NUM_CLASSES].
    labels: Labels tensor, int32 - [batch_size], with values in the
      range [0, NUM_CLASSES).
  Returns:
    A scalar int32 tensor with the number of examples (out of batch_size)
    that were predicted correctly.
  """
    # For a classifier model, we can use the in_top_k Op.
    # It returns a bool tensor with shape [batch_size] that is true for
    # the examples where the label is in the top k (here k=1)
    # of all logits for that example.
    correct = tf.nn.in_top_k(logits, labels, 1)
    # Return the number of true entries.
    accuracy = tf.reduce_sum(tf.cast(correct, tf.int32))
    return accuracy

In [8]:
def placeholder_inputs(batch_size):
    """Generate placeholder variables to represent the input tensors.
  These placeholders are used as inputs by the rest of the model building
  code and will be fed from the downloaded data in the .run() loop, below.
  Args:
    batch_size: The batch size will be baked into both placeholders.
  Returns:
    images_placeholder: Images placeholder.
    labels_placeholder: Labels placeholder.
  """
    # Note that the shapes of the placeholders match the shapes of the full
    # image and label tensors, except the first dimension is now batch_size
    # rather than the full size of the train or test data sets.
    images_placeholder = tf.placeholder(tf.float32, shape=(None,
                                                     IMAGE_PIXELS), name = "I")
    labels_placeholder = tf.placeholder(tf.int32, shape=(None))
    return images_placeholder, labels_placeholder


def fill_feed_dict(dataX, dataY, images_pl, labels_pl):
    """Fills the feed_dict for training the given step.
    A feed_dict takes the form of:
    feed_dict = {
      <placeholder>: <tensor of values to be passed for placeholder>,
      ....
    }
    Args:
    data_set: The set of images and labels, from input_data.read_data_sets()
    images_pl: The images placeholder, from placeholder_inputs().
    labels_pl: The labels placeholder, from placeholder_inputs().
    Returns:
    feed_dict: The feed dictionary mapping from placeholders to values.
    """
    # Create the feed_dict for the placeholders filled with the next
    # `batch size` examples.
    #images_feed, labels_feed = data_set.next_batch(FLAGS.batch_size,
    #                                             FLAGS.fake_data)
    
    images_feed, labels_feed = next_batch(batch_size, dataX, dataY) 
                                          
    feed_dict = {
      images_pl: images_feed,
      labels_pl: labels_feed,
    }
    return feed_dict


def do_eval(sess,
            eval_correct,
            images_placeholder,
            labels_placeholder,
            dataX, dataY):
    """Runs one evaluation against the full epoch of data.
    Args:
    sess: The session in which the model has been trained.
    eval_correct: The Tensor that returns the number of correct predictions.
    images_placeholder: The images placeholder.
    labels_placeholder: The labels placeholder.
    data_set: The set of images and labels to evaluate, from
      input_data.read_data_sets().
    """
    # And run one epoch of eval.
    true_count = 0  # Counts the number of correct predictions.
    steps_per_epoch = dataX.shape[0] // batch_size
    num_examples = steps_per_epoch * batch_size
    for step in xrange(steps_per_epoch):
        feed_dict = fill_feed_dict(dataX, dataY, images_placeholder,
                                   labels_placeholder)
        true_count += sess.run(eval_correct, feed_dict=feed_dict)
    precision = float(true_count) / num_examples
    print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
        (num_examples, true_count, precision))


def run_training():
    """Train MNIST for a number of steps."""
    # Get the sets of images and labels for training, validation, and
    # test on MNIST.
    #data_sets = input_data.read_data_sets(FLAGS.input_data_dir, FLAGS.fake_data)

    # Tell TensorFlow that the model will be built into the default Graph.
    with tf.Graph().as_default():
        # Generate placeholders for the images and labels.
        images_placeholder, labels_placeholder_x = placeholder_inputs(batch_size)
        _, labels_placeholder_y = placeholder_inputs(batch_size)

        # Build a Graph that computes predictions from the inference model.
        logits_x = inference(images_placeholder,hidden1,hidden2,class_number_x)
        logits_y = inference(images_placeholder,hidden1,hidden2,class_number_y)

        # Add to the Graph the Ops for loss calculation.
        loss_x = lossF(logits_x, labels_placeholder_x)
        loss_y = lossF(logits_y, labels_placeholder_y)

        # Add to the Graph the Ops that calculate and apply gradients.
        train_op_x = training(loss_x, learning_rate,0)
        train_op_y = training(loss_y, learning_rate,1)

        # Add the Op to compare the logits to the labels during evaluation.
        eval_correct_x = evaluation(logits_x, labels_placeholder_x)
        eval_correct_y = evaluation(logits_y, labels_placeholder_y)
        
        _, accuracy_x = tf.metrics.accuracy(labels_placeholder_x, tf.argmax(logits_x, 1))
        accuracy_summary_x = tf.summary.scalar('accuracy_x', accuracy_x)
        
        _, accuracy_y = tf.metrics.accuracy(labels_placeholder_y, tf.argmax(logits_y, 1))
        #accuracy_summary_y = tf.summary.scalar('accuracy_y', accuracy_y)

        # Build the summary Tensor based on the TF collection of Summaries.
        summary = tf.summary.merge_all()

        # Add the variable initializer Op.
        init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

        # Create a saver for writing training checkpoints.
        saver = tf.train.Saver()

        # Create a session for running Ops on the Graph.
        sess = tf.Session()

        # Instantiate a SummaryWriter to output summaries and the Graph.
        summary_writer = tf.summary.FileWriter(log_dir+ "/train")
        test_writer = tf.summary.FileWriter(log_dir + '/test')
        summary_writer.add_graph(sess.graph)

        # And then after everything is built:

        # Run the Op to initialize the variables.
        sess.run(init)

        # Start the training loop.
        for step in xrange(max_steps):
            start_time = time.time()

            # Fill a feed dictionary with the actual set of images and labels
            # for this particular training step.
            feed_dict_x = fill_feed_dict(Xtrx, ytrx, images_placeholder,
                                     labels_placeholder_x)
            
            feed_dict_y = fill_feed_dict(Xtry, ytry, images_placeholder,
                                     labels_placeholder_y)

            # Run one step of the model.  The return values are the activations
            # from the `train_op` (which is discarded) and the `loss` Op.  To
            # inspect the values of your Ops or variables, you may include them
            # in the list passed to sess.run() and the value tensors will be
            # returned in the tuple from the call.
            _, loss_value_x, accuracy_value_x = sess.run([train_op_x, loss_x, accuracy_x],
                                   feed_dict=feed_dict_x)
            
            _, loss_value_y, accuracy_value_y = sess.run([train_op_y, loss_y, accuracy_y],
                                   feed_dict=feed_dict_y)
        
            duration = time.time() - start_time

            # Write the summaries and print an overview fairly often.
            if step % 100 == 0:
                # Print status to stdout.
                # Update the events file.
                summary_str_x = sess.run(summary, feed_dict=feed_dict_x)
                summary_writer.add_summary(summary_str_x, step)
                summary_test_x, accuracy_value_test_x = sess.run([accuracy_summary_x, accuracy_x], feed_dict = {images_placeholder: Xtex, labels_placeholder_x: ytex})
                accuracy_value_test_y = sess.run(accuracy_y, feed_dict = {images_placeholder: Xtey, labels_placeholder_y: ytey})
                test_writer.add_summary(summary_test_x, step)
                summary_writer.flush()
                print('Step %d: accuracy = %.2f (%.3f sec)' % (step, accuracy_value_x, duration))
                print('Step %d: accuracy test = %.2f (%.3f sec)' % (step, accuracy_value_test_x, duration))
                print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value_x, duration))
                print("--------------------------------------------------------------------------------------")
                print('Step %d: accuracy = %.2f (%.3f sec)' % (step, accuracy_value_y, duration))
                print('Step %d: accuracy test = %.2f (%.3f sec)' % (step, accuracy_value_test_y, duration))
                print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value_y, duration))
                print("############################################################################################")

            # Save a checkpoint and evaluate the model periodically.
            if (step + 1) % 1000 == 0 or (step + 1) == max_steps:
                checkpoint_file = os.path.join(log_dir, 'model.ckpt')
                saver.save(sess, checkpoint_file, global_step=step)
                # Evaluate against the training set.
                print('Training Data Eval:')
                do_eval(sess,
                        eval_correct_x,
                        images_placeholder,
                        labels_placeholder_x,
                        Xtrx, ytrx)
                # Evaluate against the validation set.
                # Evaluate against the test set.
                print('Test Data Eval:')
                do_eval(sess,
                        eval_correct_x,
                        images_placeholder,
                        labels_placeholder_x,
                        Xtex, ytex)
        
        output_node_x = tf.cast(tf.argmax(logits_x, 1), tf.int32)
        output_node_y = tf.cast(tf.argmax(logits_y, 1), tf.int32)
        output_xy = tf.concat([output_node_x, output_node_y], 0, name="O")
        tf.train.write_graph(sess.graph_def, '.', 'NN.pbtxt')
        saver.save(sess, './NN.ckpt')
        #correct_prediction = tf.argmax(sess.run(logits, feed_dict = {images_placeholder: [[2.80714798, -0.36167914, -1.0649991,   4.10078526, -0.44053763,  0.52955198
#,  0.41685054,  1.31442308]]}))
        print(sess.run(output_xy, feed_dict = {images_placeholder: [[2.80714798, -0.36167914, -1.0649991,   4.10078526, -0.44053763,  0.52955198
,  0.41685054,  1.31442308]]}))
                

In [9]:
run_training()

Step 0: accuracy = 0.09 (0.080 sec)
Step 0: accuracy test = 0.27 (0.080 sec)
Step 0: loss = 2.48 (0.080 sec)
--------------------------------------------------------------------------------------
Step 0: accuracy = 0.28 (0.080 sec)
Step 0: accuracy test = 0.40 (0.080 sec)
Step 0: loss = 1.55 (0.080 sec)
############################################################################################
Step 100: accuracy = 0.50 (0.005 sec)
Step 100: accuracy test = 0.56 (0.005 sec)
Step 100: loss = 0.85 (0.005 sec)
--------------------------------------------------------------------------------------
Step 100: accuracy = 0.62 (0.005 sec)
Step 100: accuracy test = 0.67 (0.005 sec)
Step 100: loss = 0.47 (0.005 sec)
############################################################################################
Step 200: accuracy = 0.63 (0.005 sec)
Step 200: accuracy test = 0.65 (0.005 sec)
Step 200: loss = 0.40 (0.005 sec)
-----------------------------------------------------------------------------

Training Data Eval:
  Num examples: 5280  Num correct: 5077  Precision @ 1: 0.9616
Test Data Eval:
  Num examples: 1312  Num correct: 1237  Precision @ 1: 0.9428
Step 2000: accuracy = 0.88 (0.005 sec)
Step 2000: accuracy test = 0.89 (0.005 sec)
Step 2000: loss = 0.03 (0.005 sec)
--------------------------------------------------------------------------------------
Step 2000: accuracy = 0.92 (0.005 sec)
Step 2000: accuracy test = 0.92 (0.005 sec)
Step 2000: loss = 0.02 (0.005 sec)
############################################################################################
Step 2100: accuracy = 0.89 (0.005 sec)
Step 2100: accuracy test = 0.89 (0.005 sec)
Step 2100: loss = 0.17 (0.005 sec)
--------------------------------------------------------------------------------------
Step 2100: accuracy = 0.92 (0.005 sec)
Step 2100: accuracy test = 0.92 (0.005 sec)
Step 2100: loss = 0.05 (0.005 sec)
############################################################################################
Step 2

Step 3900: accuracy = 0.92 (0.005 sec)
Step 3900: accuracy test = 0.93 (0.005 sec)
Step 3900: loss = 0.02 (0.005 sec)
--------------------------------------------------------------------------------------
Step 3900: accuracy = 0.94 (0.005 sec)
Step 3900: accuracy test = 0.94 (0.005 sec)
Step 3900: loss = 0.10 (0.005 sec)
############################################################################################
Training Data Eval:
  Num examples: 5280  Num correct: 5214  Precision @ 1: 0.9875
Test Data Eval:
  Num examples: 1312  Num correct: 1274  Precision @ 1: 0.9710
Step 4000: accuracy = 0.93 (0.008 sec)
Step 4000: accuracy test = 0.93 (0.008 sec)
Step 4000: loss = 0.02 (0.008 sec)
--------------------------------------------------------------------------------------
Step 4000: accuracy = 0.95 (0.008 sec)
Step 4000: accuracy test = 0.95 (0.008 sec)
Step 4000: loss = 0.05 (0.008 sec)
############################################################################################
Step 4

Step 5800: accuracy = 0.94 (0.006 sec)
Step 5800: accuracy test = 0.94 (0.006 sec)
Step 5800: loss = 0.02 (0.006 sec)
--------------------------------------------------------------------------------------
Step 5800: accuracy = 0.96 (0.006 sec)
Step 5800: accuracy test = 0.96 (0.006 sec)
Step 5800: loss = 0.02 (0.006 sec)
############################################################################################
Step 5900: accuracy = 0.94 (0.004 sec)
Step 5900: accuracy test = 0.94 (0.004 sec)
Step 5900: loss = 0.02 (0.004 sec)
--------------------------------------------------------------------------------------
Step 5900: accuracy = 0.96 (0.004 sec)
Step 5900: accuracy test = 0.96 (0.004 sec)
Step 5900: loss = 0.02 (0.004 sec)
############################################################################################
Training Data Eval:
  Num examples: 5280  Num correct: 5241  Precision @ 1: 0.9926
Test Data Eval:
  Num examples: 1312  Num correct: 1278  Precision @ 1: 0.9741
Step 6

Step 7700: accuracy = 0.95 (0.005 sec)
Step 7700: accuracy test = 0.95 (0.005 sec)
Step 7700: loss = 0.14 (0.005 sec)
--------------------------------------------------------------------------------------
Step 7700: accuracy = 0.96 (0.005 sec)
Step 7700: accuracy test = 0.96 (0.005 sec)
Step 7700: loss = 0.01 (0.005 sec)
############################################################################################
Step 7800: accuracy = 0.95 (0.005 sec)
Step 7800: accuracy test = 0.95 (0.005 sec)
Step 7800: loss = 0.01 (0.005 sec)
--------------------------------------------------------------------------------------
Step 7800: accuracy = 0.96 (0.005 sec)
Step 7800: accuracy test = 0.96 (0.005 sec)
Step 7800: loss = 0.01 (0.005 sec)
############################################################################################
Step 7900: accuracy = 0.95 (0.005 sec)
Step 7900: accuracy test = 0.95 (0.005 sec)
Step 7900: loss = 0.01 (0.005 sec)
--------------------------------------------------

...

Step 11500: accuracy = 0.96 (0.005 sec)
Step 11500: accuracy test = 0.96 (0.005 sec)
Step 11500: loss = 0.00 (0.005 sec)
--------------------------------------------------------------------------------------
Step 11500: accuracy = 0.97 (0.005 sec)
Step 11500: accuracy test = 0.97 (0.005 sec)
Step 11500: loss = 0.01 (0.005 sec)
############################################################################################
Step 11600: accuracy = 0.96 (0.006 sec)
Step 11600: accuracy test = 0.96 (0.006 sec)
Step 11600: loss = 0.05 (0.006 sec)
--------------------------------------------------------------------------------------
Step 11600: accuracy = 0.97 (0.006 sec)
Step 11600: accuracy test = 0.97 (0.006 sec)
Step 11600: loss = 0.01 (0.006 sec)
############################################################################################
Step 11700: accuracy = 0.96 (0.008 sec)
Step 11700: accuracy test = 0.96 (0.008 sec)
Step 11700: loss = 0.02 (0.008 sec)
-----------------------------------

Step 13400: accuracy = 0.97 (0.005 sec)
Step 13400: accuracy test = 0.97 (0.005 sec)
Step 13400: loss = 0.00 (0.005 sec)
--------------------------------------------------------------------------------------
Step 13400: accuracy = 0.97 (0.005 sec)
Step 13400: accuracy test = 0.97 (0.005 sec)
Step 13400: loss = 0.01 (0.005 sec)
############################################################################################
Step 13500: accuracy = 0.97 (0.006 sec)
Step 13500: accuracy test = 0.97 (0.006 sec)
Step 13500: loss = 0.00 (0.006 sec)
--------------------------------------------------------------------------------------
Step 13500: accuracy = 0.97 (0.006 sec)
Step 13500: accuracy test = 0.97 (0.006 sec)
Step 13500: loss = 0.00 (0.006 sec)
############################################################################################
Step 13600: accuracy = 0.97 (0.006 sec)
Step 13600: accuracy test = 0.97 (0.006 sec)
Step 13600: loss = 0.02 (0.006 sec)
-----------------------------------

Step 15300: accuracy = 0.97 (0.006 sec)
Step 15300: accuracy test = 0.97 (0.006 sec)
Step 15300: loss = 0.00 (0.006 sec)
--------------------------------------------------------------------------------------
Step 15300: accuracy = 0.98 (0.006 sec)
Step 15300: accuracy test = 0.98 (0.006 sec)
Step 15300: loss = 0.00 (0.006 sec)
############################################################################################
Step 15400: accuracy = 0.97 (0.006 sec)
Step 15400: accuracy test = 0.97 (0.006 sec)
Step 15400: loss = 0.01 (0.006 sec)
--------------------------------------------------------------------------------------
Step 15400: accuracy = 0.98 (0.006 sec)
Step 15400: accuracy test = 0.98 (0.006 sec)
Step 15400: loss = 0.01 (0.006 sec)
############################################################################################
Step 15500: accuracy = 0.97 (0.005 sec)
Step 15500: accuracy test = 0.97 (0.005 sec)
Step 15500: loss = 0.00 (0.005 sec)
-----------------------------------

Step 17200: accuracy = 0.97 (0.006 sec)
Step 17200: accuracy test = 0.97 (0.006 sec)
Step 17200: loss = 0.01 (0.006 sec)
--------------------------------------------------------------------------------------
Step 17200: accuracy = 0.98 (0.006 sec)
Step 17200: accuracy test = 0.98 (0.006 sec)
Step 17200: loss = 0.00 (0.006 sec)
############################################################################################
Step 17300: accuracy = 0.97 (0.005 sec)
Step 17300: accuracy test = 0.97 (0.005 sec)
Step 17300: loss = 0.00 (0.005 sec)
--------------------------------------------------------------------------------------
Step 17300: accuracy = 0.98 (0.005 sec)
Step 17300: accuracy test = 0.98 (0.005 sec)
Step 17300: loss = 0.00 (0.005 sec)
############################################################################################
Step 17400: accuracy = 0.97 (0.006 sec)
Step 17400: accuracy test = 0.97 (0.006 sec)
Step 17400: loss = 0.00 (0.006 sec)
-----------------------------------

Step 19100: accuracy = 0.98 (0.006 sec)
Step 19100: accuracy test = 0.98 (0.006 sec)
Step 19100: loss = 0.00 (0.006 sec)
--------------------------------------------------------------------------------------
Step 19100: accuracy = 0.98 (0.006 sec)
Step 19100: accuracy test = 0.98 (0.006 sec)
Step 19100: loss = 0.00 (0.006 sec)
############################################################################################
Step 19200: accuracy = 0.98 (0.005 sec)
Step 19200: accuracy test = 0.98 (0.005 sec)
Step 19200: loss = 0.01 (0.005 sec)
--------------------------------------------------------------------------------------
Step 19200: accuracy = 0.98 (0.005 sec)
Step 19200: accuracy test = 0.98 (0.005 sec)
Step 19200: loss = 0.06 (0.005 sec)
############################################################################################
Step 19300: accuracy = 0.98 (0.008 sec)
Step 19300: accuracy test = 0.98 (0.008 sec)
Step 19300: loss = 0.00 (0.008 sec)
-----------------------------------

In [10]:
MODEL_NAME = 'NN'

# Freeze the graph

input_graph_path = MODEL_NAME+'.pbtxt'
checkpoint_path = './'+MODEL_NAME+'.ckpt'
input_saver_def_path = ""
input_binary = False
output_node_names = "O"
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_name = 'frozen_'+MODEL_NAME+'.pb'
output_optimized_graph_name = 'optimized_'+MODEL_NAME+'.pb'
clear_devices = True


freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                          input_binary, checkpoint_path, output_node_names,
                          restore_op_name, filename_tensor_name,
                          output_frozen_graph_name, clear_devices, "")

INFO:tensorflow:Restoring parameters from ./NN.ckpt
INFO:tensorflow:Froze 12 variables.
Converted 12 variables to const ops.


In [11]:
# Optimizacion

input_graph_def = tf.GraphDef()
with tf.gfile.Open(output_frozen_graph_name, "rb") as f:
    data = f.read()
    input_graph_def.ParseFromString(data)

output_graph_def = optimize_for_inference_lib.optimize_for_inference(
        input_graph_def,
        ["I"], # an array of the input node(s)
        ["O"], # an array of output nodes
        tf.float32.as_datatype_enum)

# Save the optimized graph

f = tf.gfile.FastGFile(output_optimized_graph_name, "w")
f.write(output_graph_def.SerializeToString())